1. SVM, Linear, Decision Tree do stacking
2. NNIST 0~9 手寫辨識 CNN + LSTM 做成圖表的架構
3. LSTM . RNN

# 1

In [ ]:
import numpy as np
import pandas as pd
from sklearn import datasets
from sklearn.model_selection import train_test_split, KFold
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import StackingClassifier
from sklearn.metrics import accuracy_score
import torch
import torch.nn as nn
import torch.optim as optim

# 載入 Iris 資料集
iris = datasets.load_iris()
X = iris.data
y = iris.target

# 分割資料集
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# 定義基模型
base_models = [
    ('svm', SVC(probability=True)),
    ('lr', LogisticRegression(max_iter=1000)),
    ('dt', DecisionTreeClassifier())
]

# 定義元模型
class MetaModel(nn.Module):
    def __init__(self, input_dim):
        super(MetaModel, self).__init__()
        self.fc = nn.Linear(input_dim, 3)  # 3 classes for Iris dataset

    def forward(self, x):
        return self.fc(x)

# 訓練基模型並生成其預測結果
def train_base_models(base_models, X_train, y_train, X_val, y_val):
    predictions = np.zeros((X_val.shape[0], len(base_models)))
    for i, (name, model) in enumerate(base_models):
        model.fit(X_train, y_train)
        predictions[:, i] = model.predict(X_val)
    return predictions

# 使用 K-fold 交叉驗證來生成基模型的預測結果
kf = KFold(n_splits=5, shuffle=True, random_state=42)
meta_features = np.zeros((X_train.shape[0], len(base_models)))

for train_index, val_index in kf.split(X_train):
    X_train_fold, X_val_fold = X_train[train_index], X_train[val_index]
    y_train_fold, y_val_fold = y_train[train_index], y_train[val_index]
    meta_features[val_index] = train_base_models(base_models, X_train_fold, y_train_fold, X_val_fold, y_val_fold)

# 訓練元模型
meta_model = MetaModel(input_dim=len(base_models))
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(meta_model.parameters(), lr=0.01)

# 將 meta_features 和 y_train 轉換為 PyTorch 張量
meta_features_tensor = torch.tensor(meta_features, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train, dtype=torch.long)

# 訓練元模型
num_epochs = 100
for epoch in range(num_epochs):
    meta_model.train()
    optimizer.zero_grad()
    outputs = meta_model(meta_features_tensor)
    loss = criterion(outputs, y_train_tensor)
    loss.backward()
    optimizer.step()

# 使用基模型對測試集進行預測
test_meta_features = train_base_models(base_models, X_train, y_train, X_test, y_test)

# 使用元模型對測試集進行最終預測
meta_model.eval()
test_meta_features_tensor = torch.tensor(test_meta_features, dtype=torch.float32)
with torch.no_grad():
    test_outputs = meta_model(test_meta_features_tensor)
    _, test_preds = torch.max(test_outputs, 1)

# 計算測試集上的準確率
accuracy = accuracy_score(y_test, test_preds.numpy())
print(f'Test Accuracy: {accuracy:.4f}')


In [ ]:
import numpy as np
import pandas as pd
from sklearn import datasets
from sklearn.model_selection import train_test_split, StratifiedKFold
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import StackingClassifier
from sklearn.metrics import accuracy_score
import matplotlib.pyplot as plt

# 1. 載入資料集
iris = datasets.load_iris()
X = iris.data
y = iris.target

# 2. 資料前處理
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42, stratify=y)

# 3. 訓練第一層模型
base_models = [
    ('svm', SVC(probability=True, random_state=42)),
    ('lr', LogisticRegression(random_state=42)),
    ('dt', DecisionTreeClassifier(random_state=42))
]

# 4. 訓練第二層模型（元模型）
meta_model = LogisticRegression(random_state=42)

# 5. 建立 Stacking 模型
stacking_model = StackingClassifier(estimators=base_models, final_estimator=meta_model, cv=5)

# 6. 訓練 Stacking 模型
stacking_model.fit(X_train, y_train)

# 7. 評估模型性能
y_pred = stacking_model.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print(f'Stacking Model Accuracy: {accuracy:.4f}')

# 8. 可視化結果
def plot_decision_boundary(model, X, y, title):
    x_min, x_max = X[:, 0].min() - 1, X[:, 0].max() + 1
    y_min, y_max = X[:, 1].min() - 1, X[:, 1].max() + 1
    xx, yy = np.meshgrid(np.arange(x_min, x_max, 0.01),
                         np.arange(y_min, y_max, 0.01))
    Z = model.predict(np.c_[xx.ravel(), yy.ravel()])
    Z = Z.reshape(xx.shape)
    plt.contourf(xx, yy, Z, alpha=0.8)
    plt.scatter(X[:, 0], X[:, 1], c=y, edgecolor='k', marker='o')
    plt.title(title)
    plt.show()

# 只用前兩個特徵做可視化
X_train_vis = X_train[:, :2]
X_test_vis = X_test[:, :2]

for name, model in base_models:
    model.fit(X_train_vis, y_train)
    plot_decision_boundary(model, X_test_vis, y_test, f'{name} Decision Boundary')

# 可視化 Stacking 模型
stacking_model.fit(X_train_vis, y_train)
plot_decision_boundary(stacking_model, X_test_vis, y_test, 'Stacking Decision Boundary')


# 2

In [ ]:
import tensorflow as tf
from tensorflow.keras import layers, models

# 加載MNIST數據集
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.mnist.load_data()

# 將數據標準化到0-1之間
x_train = x_train.astype('float32') / 255.0
x_test = x_test.astype('float32') / 255.0

# CNN模型
cnn_input = layers.Input(shape=(28, 28, 1))
x = layers.Conv2D(32, (3, 3), activation='relu')(cnn_input)
x = layers.MaxPooling2D((2, 2))(x)
x = layers.Conv2D(64, (3, 3), activation='relu')(x)
x = layers.MaxPooling2D((2, 2))(x)
cnn_output = layers.Flatten()(x)

# LSTM模型
lstm_input = layers.Input(shape=(28, 28))
y = layers.LSTM(128)(lstm_input)
lstm_output = y

# 特徵融合
merged = layers.concatenate([cnn_output, lstm_output])

# 全連接層
z = layers.Dense(128, activation='relu')(merged)
z = layers.Dense(10, activation='softmax')(z)

# 定義最終模型
model = models.Model(inputs=[cnn_input, lstm_input], outputs=z)

# 編譯模型
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# 將輸入數據適配模型的輸入格式
x_train_cnn = x_train[..., tf.newaxis]  # 增加一個維度以適應CNN輸入
x_train_lstm = x_train  # 保持原始形狀以適應LSTM輸入
x_test_cnn = x_test[..., tf.newaxis]
x_test_lstm = x_test

# 訓練模型
model.fit([x_train_cnn, x_train_lstm], y_train, epochs=10, batch_size=64, validation_data=([x_test_cnn, x_test_lstm], y_test))

# 評估模型
test_loss, test_acc = model.evaluate([x_test_cnn, x_test_lstm], y_test)
print(f'Test accuracy: {test_acc}')
